In [2]:
#Install kaggle
!pip install -q kaggle

In [3]:
#Upload kaggle.json from local systema
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rubinshrivastava","key":"87891af05c5a43eb394be202a23effed"}'}

In [4]:
#Create a kaggle folder
! mkdir ~/.kaggle

In [5]:
#Copy the kaggle.json to folder created
! cp kaggle.json ~/.kaggle/

In [6]:
#Permission for the json to act
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets download -d yelp-dataset/yelp-dataset

100% 4.07G/4.07G [02:14<00:00, 49.9MB/s]
100% 4.07G/4.07G [02:14<00:00, 32.5MB/s]


In [8]:
! unzip yelp-dataset.zip

Archive:  yelp-dataset.zip
  inflating: Dataset_User_Agreement.pdf  
  inflating: yelp_academic_dataset_business.json  
  inflating: yelp_academic_dataset_checkin.json  
  inflating: yelp_academic_dataset_review.json  
  inflating: yelp_academic_dataset_tip.json  
  inflating: yelp_academic_dataset_user.json  


In [9]:
!pip install -q findspark

In [10]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [11]:
#Start a Spark session
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

spark = SparkSession.builder.appName("DataFramesJson").getOrCreate()

In [12]:
#Read reviews file into a pyspark df
yelp_reviews = spark.read.json('yelp_academic_dataset_review.json')

In [13]:
yelp_reviews.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [14]:
#Read businesses file intro a dataframe
businesses_df = spark.read.json('yelp_academic_dataset_business.json')

#Filter only categories related to restaurants in the business_df
restaurants_df = businesses_df.where(businesses_df.categories.like('%Restaurants%'))

In [15]:
restaurants_df.show()

+--------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|        city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|         935 Race St|{null, null, u'no...|MTSW4McQd7CbVtyjq...|Restaurants, Food...|Philadelphia|{7:0-21:0, 7:0-20...|      1|   39.9555052|   -75.1555641|  St Honore Pastries|      19107|          80|  4.0|   PA|
|       615 S Main St|{null, null, u'no...|CF33F8-E6oudUQ46H...|Burgers, Fast Foo...|Ashland City|{9:0-0:0, 0:0-0:0...|      1|    36.269593

In [16]:
#Filter businesses_df only for the city of Philadelphia ordering in descendent by review counts 
#Ph_businesses = restaurants_df.where(businesses_df.city == 'Philadelphia')
Ph_businesses = restaurants_df.where(businesses_df.city == 'Philadelphia').orderBy( restaurants_df.review_count.desc())

#Rename columns in order to join with yelp_reviews dataframe
Ph_businesses = Ph_businesses.withColumnRenamed('business_id','ID' )
Ph_businesses = Ph_businesses.withColumnRenamed('stars','stars_business' )

#Drop attributes column
Ph_businesses = Ph_businesses.drop('attributes','hours','is_open','state')

In [ ]:
Ph_businesses.show()

+-------------------+--------------------+--------------------+------------+-------------+--------------+--------------------+-----------+------------+--------------+
|            address|                  ID|          categories|        city|     latitude|     longitude|                name|postal_code|review_count|stars_business|
+-------------------+--------------------+--------------------+------------+-------------+--------------+--------------------+-----------+------------+--------------+
|       51 N 12th St|ytynqOUb3hjKeJfRj...|Candy Stores, Sho...|Philadelphia|39.9533414645|-75.1588545174|Reading Terminal ...|      19107|        5721|           4.5|
|1237 E Passyunk Ave|PP3BBaVxZLcJU54uP...|Italian, American...|Philadelphia|39.9332010583| -75.159265814|Pat's King of Steaks|      19147|        4250|           3.0|
|      1219 S 9th St|IkY2ticzHEn4QFn8h...|Sandwiches, Chees...|Philadelphia| 39.933836785|-75.1588137524|       Geno's Steaks|      19147|        3401|           2.5

In [17]:
#Get the schema of the Philadelphia df
Ph_businesses.printSchema()

root
 |-- address: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- stars_business: double (nullable = true)



In [18]:
#Save a dataframe into a file with unique restaurant ID, to use for location 

#Create a window function and patitionby ID column
w2 = Window.partitionBy('ID').orderBy(col('ID'))
#Add a new row column to use as an index using the function row_number, apply it on the partition by and filter first 10 rows.
#Unique_restaurants = Ph_businesses.withColumn('row',row_number().over(w2)).filter(col('row')==1).drop("row")
Unique_restaurants = Ph_businesses.withColumn('row',row_number().over(w2)).filter(col('row')==1).orderBy( Ph_businesses.review_count.desc() ).drop("row")
Unique_restaurants = Unique_restaurants.limit(1000)

In [19]:
Unique_restaurants.count()

1000

In [20]:
Unique_restaurants.show()

+-------------------+--------------------+--------------------+------------+-------------+--------------+--------------------+-----------+------------+--------------+
|            address|                  ID|          categories|        city|     latitude|     longitude|                name|postal_code|review_count|stars_business|
+-------------------+--------------------+--------------------+------------+-------------+--------------+--------------------+-----------+------------+--------------+
|       51 N 12th St|ytynqOUb3hjKeJfRj...|Candy Stores, Sho...|Philadelphia|39.9533414645|-75.1588545174|Reading Terminal ...|      19107|        5721|           4.5|
|1237 E Passyunk Ave|PP3BBaVxZLcJU54uP...|Italian, American...|Philadelphia|39.9332010583| -75.159265814|Pat's King of Steaks|      19147|        4250|           3.0|
|      1219 S 9th St|IkY2ticzHEn4QFn8h...|Sandwiches, Chees...|Philadelphia| 39.933836785|-75.1588137524|       Geno's Steaks|      19147|        3401|           2.5

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
#Save dataframe to csv
Unique_restaurants.write.options(header=True, delimiter=",").csv('/drive/folders/1ZhyMoKRl1pXF0kDIPjAPXnjwMVQ9rm1N?usp=sharing/restaurent.csv', mode='overwrite')

In [22]:
#Get the schema of the yelp reviews df
yelp_reviews.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



#Join the yelp_reviews and Ph_businesses dataframes using the ID and business_id as the matching field to get as many reviews per restaurant as possible
#combined_data = Ph_businesses.join(yelp_reviews, Ph_businesses.ID == yelp_reviews.business_id, how='left')

In [23]:
combined_data = Ph_businesses.join(yelp_reviews, Ph_businesses.ID == yelp_reviews.business_id, how='left')
combined_data.show()

+--------------------+--------------------+--------------------+------------+------------+-------------+----------------+-----------+------------+--------------+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|             address|                  ID|          categories|        city|    latitude|    longitude|            name|postal_code|review_count|stars_business|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+--------------------+--------------------+------------+------------+-------------+----------------+-----------+------------+--------------+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|4040 City Ave, Bl...|-fjIX9bvBKwXjQWfd...|Breakfast & Brunc...|Philadelphia|40.007403796|-75.212711425|Real Food Eatery|      191

In [24]:
#https://sparkbyexamples.com/pyspark/pyspark-select-first-row-of-each-group/
#To select a limited number of rows per ID

#Create a window function and patitionby ID column
w2 = Window.partitionBy('ID').orderBy(col('ID'))

#Add a new row column to use as an index using the function row_number, apply it on the partition by and filter first 10 rows.
limited_df = combined_data.withColumn('row',row_number().over(w2)).filter(col('row')<=10).orderBy(combined_data.review_count.desc()).limit(10000)
limited_df.show()

+-------------------+--------------------+--------------------+------------+-------------+--------------+--------------------+-----------+------------+--------------+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+---+
|            address|                  ID|          categories|        city|     latitude|     longitude|                name|postal_code|review_count|stars_business|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|row|
+-------------------+--------------------+--------------------+------------+-------------+--------------+--------------------+-----------+------------+--------------+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+---+
|       51 N 12th St|ytynqOUb3hjKeJfRj...|Candy Stores, Sho...|Philadelphia|39.9533414645|-75.158854517

In [25]:
#Count the total of rows in limited df
limited_df.count()

10000

In [26]:
#Drop columns that are not needed
cleaned_df = limited_df.drop('address','attributes','hours','is_open','postal_code','state','business_id','cool','date','funny','useful','row')

#Rename ambiguous columns
cleaned_df = cleaned_df.withColumnRenamed('ID','business_id' )
cleaned_df = cleaned_df.withColumnRenamed('stars','stars_review' ) 

cleaned_df.show()

+--------------------+--------------------+------------+-------------+--------------+--------------------+------------+--------------+--------------------+------------+--------------------+--------------------+
|         business_id|          categories|        city|     latitude|     longitude|                name|review_count|stars_business|           review_id|stars_review|                text|             user_id|
+--------------------+--------------------+------------+-------------+--------------+--------------------+------------+--------------+--------------------+------------+--------------------+--------------------+
|ytynqOUb3hjKeJfRj...|Candy Stores, Sho...|Philadelphia|39.9533414645|-75.1588545174|Reading Terminal ...|        5721|           4.5|kKC5pBPkUCWo6mKYF...|         5.0|I love this place...|mGnZFbk2gqLLtGW-m...|
|ytynqOUb3hjKeJfRj...|Candy Stores, Sho...|Philadelphia|39.9533414645|-75.1588545174|Reading Terminal ...|        5721|           4.5|sFQrhTbTah0o2kU_P...| 

In [35]:
#Erase from column 'text' \n, \t, ','
#https://sparkbyexamples.com/pyspark/pyspark-replace-column-values/#:~:text=By%20using%20PySpark%20SQL%20function,Road%20string%20on%20address%20column.
from pyspark.sql.functions import regexp_replace, when

cleaned_df = cleaned_df.withColumn('text',
                       when(cleaned_df.text.contains('\n'),regexp_replace(cleaned_df.text,'\n',';')) \
                      .when(cleaned_df.text.contains('\t'),regexp_replace(cleaned_df.text,'\t',';')) \
                      .when(cleaned_df.text.contains(','),regexp_replace(cleaned_df.text,',',';')) \
                      .when(cleaned_df.text.contains('\\'),regexp_replace(cleaned_df.text,'\\',';')) \
                      .when(cleaned_df.text.contains('\s'),regexp_replace(cleaned_df.text,'\s',';')))

In [36]:
cleaned_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- stars_business: double (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars_review: double (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: string (nullable = true)



making csv

In [37]:
#Store PySpark dataframe to csv file
#lim2.write.option("header",True).csv('csv/', mode='overwrite')
cleaned_df.write.options(header=True, delimiter=",").csv('csv2/', mode='overwrite')
